In [6]:
# %load network.py
import random
import numpy as np

class Network(object):
    # Defines the network as a class that takes a list as its only argument.
    # Attributes of the network
    def __init__(self, sizes):
        self.num_layers = len(sizes) # Takes the lenght of list 'sizes' to save the number of layers in the network
        self.sizes = sizes  # Saves the list 'sizes' that contains the number of neurons in each layer
        
        # Sets random biases for all neurons in the network except the input layer, and saves them in a list of lists
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]  
        
        # Sets random weights for all the connections between neurons and saves them in a list of matrices of 
        # y rows and x columns (given a layer "l" of 'y' neurons and its immediate previous layer "l-1" with 'x' neurons)
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]


    # Returns the output of the network
    def feedforward(self, a):
        # Calculates the outputs of the last layer by feedingforwarding the network once
        for b, w in zip(self.biases, self.weights): 
            # Uses the dot product of numpy library to multiply matrices (w) and column vectors (a)
            a = sigmoid(np.dot(w, a)+b)
        ''' softmax layer '''
        b = softmax(a)
        return b


    
    # Function that trains the network with the SGD method. 
    '''beta parameter added for SGD with momentum'''
    def SGDM(self, training_data, epochs, mini_batch_size, eta, beta, 
            test_data=None):

        # Takes the training data and saves its size
        training_data = list(training_data)
        n = len(training_data)

        # Saves the test data and its size, in the case that test data is provided
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        # Iterates over the total number of epochs
        for j in range(epochs):
            # First, it sorts the data in random order. Then, creates a list of all minibatches of size mini_batch_size
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            # Updates the weights and biases with update_mini_batch function after every minibatch
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, beta)
                
            # In case the test data is provided, test results (provided by evaluate function) will be compared to
            # the total number of data in test data. Otherwise, it just informs about the completion of an epoch
            if test_data:
                print("Epoch {} : {} / {}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {} complete".format(j))

                
    
    # Updates the biases and weights for every minibatch
    '''beta parameter added for SGD with momentum'''
    def update_mini_batch(self, mini_batch, eta, beta):
        # Lists that contains the derivatives with respect to biases and weights, initially these are set to 0
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        ''' lists to store the momentum terms in each update of the bias and weights '''
        momentum_b = [np.zeros(b.shape) for b in self.biases]
        momentum_w = [np.zeros(w.shape) for w in self.weights]
        
        # Applies backpropagation to a minibatch
        for x, y in mini_batch:
            # Saves the results of backpropagation
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            
            # Updates the value of the gradients adding to the existing gradients the new delta_nabla for b and w
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        '''updating the values of the "momentum"for both weights and biases '''
        momentum_b = [(beta*mb)+((1-beta)*nb) for mb, nb in zip(momentum_b, nabla_b)]
        momentum_w = [(beta*mw)+((1-beta)*nw) for mw, nw in zip(momentum_w, nabla_w)]
        
        # Then, updates the values of weights and biases according to
        #  w ---> w' = w-(eta/m) \partial{C_x}/\partial{w}
        #  b ---> b' = b-(eta/m) \partial{C_x}/\partial{b}
        self.weights = [w-(eta/len(mini_batch))*mw for w, mw in zip(self.weights, momentum_w)]
        self.biases = [b-(eta/len(mini_batch))*mb for b, mb in zip(self.biases, momentum_b)]
    
        
        
    #### Backpropagation algorithm
    def backprop(self, x, y):
        # Gradients, initially set  to 0
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        ## Feedforward: initializes the network once
        activation = x    # Set the initial activation as the inputs from the training data in the minibatches
        activations = [x]    # Stores all the activations, layer by layer. First layer is just the input data
        zs = []    # list to store all the z (argument of activation function) vectors, layer by layer
        
        # Iteration over all the layers
        for b, w in zip(self.biases, self.weights):
            # Generates the arguments for the sigmoid/activation function of every neuron in a layer and stores them
            z = np.dot(w, activation)+b
            zs.append(z)
            # Generates the activations of the neurons in the layer using the z's calculated before and stores them
            activation = sigmoid(z)
            activations.append(activation)
        
        '''Softmax layer:
           Outputs a vector of probabilities given by the softmax 
           function applied to the network output activations'''
        probs = softmax(activations[-1])
        
        ## Backward pass: Actual training of the network
        # Calculates the vector of "errors" (delta) of the last layer
        '''Cross-Entropy only changes the form of delta in the last 'layer' (softmax), that is
            delta = softmax(z) - y. The true values 'y' are already one-hot encoded'''
        delta = self.cost_derivative(probs, y)
        
        # Stores the gradient for the last layer, remembering that 
        # \partial(C)/\partial{b} = delta
        # \partial(C)/\partial{w} = (delta)a^T
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Iterates over the layers between the last and input layers. The loop starts at 2, because inside the 
        # loop the index is inverted, that is, it actually starts with index [-2], the penultimate layer, 
        # all the way to the layer before the input layer
        for l in range(2, self.num_layers):
            z = zs[-l]    # The values of the neurons in the a layer
            sp = sigmoid_prime(z)    # The values of the derivative of the sigmoid function for that layer
            
            # Computes deltas, in this case given as ----> delta = [(w)^T(delta)](sigmoid'(z))
            # delta updates backwards
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp 
            
            # Stores the gradients for the same layer
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    

    # Gives the number of inputs for which the network gives a correct result using the test data
    def evaluate(self, test_data):
        # Feeds the once trained network with the test data, then stores the output of the network (identified as the
        # index of the neuron with the maximum activation value) along with the correct result specified in test_data
        # Then sums each result and returns the total sum (a correct result is identified as 1 and an incorrect one as 0)
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        ''' Gives the derivatives of C_x (Cross-Entropy) of the output activations
           \partial{C_x}/\partial{z} = a-y '''
        return (output_activations-y)


def softmax(z):
    '''Returns a vector to which softmax function has been applied to each one of its elements'''
    exp = np.round(np.exp(z), 3)
    return exp/sum(exp)
    
def sigmoid(z):
    # Returns the value of the sigmoid function
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Returns the value of the derivative of sigmoid function
    return sigmoid(z)*(1-sigmoid(z))